# DATA SCIENTIST HIRE

### Task: Predict The Probability of Interview to accept a job offer at a Big Data Company(Not Mentioned). Employing The  applications of Machine Learning Give Likely Reasons why Candidates May choose not to work at the company

In [ ]:
#Import Needed Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict,GridSearchCV,StratifiedShuffleSplit
from sklearn.metrics import precision_score,recall_score,roc_auc_score,confusion_matrix,roc_curve
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier,plot_tree
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [ ]:
#Load Data
train_data=pd.read_csv('/kaggle/input/hr-analytics-job-change-of-data-scientists/aug_train.csv',na_values='NULL')
test_data=pd.read_csv('/kaggle/input/hr-analytics-job-change-of-data-scientists/aug_test.csv',na_values='NULL')
sample_submission=pd.read_csv('/kaggle/input/hr-analytics-job-change-of-data-scientists/sample_submission.csv',na_values='NULL')


In [ ]:
train_data[:5]

In [ ]:
id=test_data.enrollee_id.values
print('Train_Data_Shape: ',train_data.shape,'\n\n','Test_Data_Shape: ',test_data.shape,'\n\n','Sample_Submission_Shape: ',sample_submission.shape)

In [ ]:
train_data.info()

There are a good number of Missing values as can be seen above we have to devise ways to deal with them at 

In [ ]:
#Check for Duplicates in Train and Test Data
print('Train_Data: ',train_data.enrollee_id.duplicated().sum(),'Duplicates')
print('Test_Data: ',test_data.enrollee_id.duplicated().sum(),'Duplicates')

In [ ]:
#Combine train and test data for analysis
y_train=train_data.target
data=train_data[train_data.columns[:-1]].append(test_data)
data.shape

In [ ]:
data.info()

In [ ]:
#Function To Calculate Number of Missing Datapoints for each rows
def missing(x,col):
    print('Missing Values in',col,': ',(x.shape[0]-x[col].notnull().sum()))

In [ ]:
data.head(5)

## FEATURE ANALYSIS AND VISUALIZATION

In [ ]:
missing(data,'training_hours')

In [ ]:
data.training_hours.describe()

In [ ]:
#Plot of Training_Hours
print('Fig 1.0')
plt.figure(figsize=(8,10))
plt.title('Training_Hours_Plot',size=20)
plt.xlabel('Training_Hours',size=15)
plt.hist(data.training_hours,color='g')
plt.grid('True')
plt.show()

Majority of Candidates as can be seen from fig 1.0 spent less than 100 Hours on Training

In [ ]:
missing(data,'last_new_job')

In [ ]:
data.last_new_job.value_counts()

In [ ]:
#Chart of New Jobs
print('Fig 1.1')
def piechart(col,x):
    " piechart(col,size);\n 'col' represents the column to be plotted it shld be entered as a string object; 'size' represents the desired figure size entered as a tuple example (5,10)"
    plt.figure(figsize=x)
    plt.title(col.upper()+'_CHART',size=20)
    plt.pie(x=data[col].value_counts().values,explode=[0.05 for x in range(len(data[col].value_counts().values))],autopct='%.2f',labels=data[col].value_counts().index)
    plt.legend(title=col,loc='best')
    plt.show()
piechart('last_new_job',(10,10))

Fig 1.1 Above is a plot of The distribution of the number of jobs previously held by candidates, Most Candidates have had just 1(One) previous job before applying to the company this represents 42% of the total candidates while candidates who have had more than 4 jobs represents 17% of the total candidates, some 13% of candidates have never had a previous job. It will be interesting to see if work experience influences candidates decision to accept an offer at the company

In [ ]:
# Replace 'never' with 0, '>4' with 5 then fill missing with one
data.last_new_job.replace(['never','>4'],[0,5],inplace=True)
data.last_new_job.fillna('1',inplace=True)
data.last_new_job.astype('int64')
data.last_new_job.value_counts()

In [ ]:
missing(data,'company_type')

Company type has a lot of missing values

In [ ]:
#Pie Plot Of Company Type
print('Fig 1.2')
piechart('company_type',(15,15))

Fig 1.2 is a plot of Comapany type feature it has  a low variance that implies that few classes occurs many time PVTLTD has 75% of the data points this can make it very uninformative and not useful to our target

In [ ]:
#We will fill The missing values in company type with Pvt Ltd
data.company_type.fillna('Pvt Ltd',inplace=True)
data.company_type.replace(data.company_type.value_counts().index,[0,1,2,3,4,5],inplace=True)
data.company_type.value_counts()

In [ ]:
#missing values
missing(data,'company_size')

In [ ]:
#Plot of Company_Size
print('Fig 1.3')
plt.figure(figsize=(10,10));
plt.title('Company_Size_Plot',size=20)
plt.xlabel('Company_size',size=15)
plt.grid('True')
plt.hist(data.company_size[data.company_size.notnull()],color='purple')
plt.show()

In [ ]:
#Replacing Values in Company Size with custom encoding
data.company_size.replace(['<10','10/49','50-99','100-500','500-999','1000-4999','5000-9999','10000+'],[x for x in range(1,9)],inplace=True)
data.company_size.fillna(-1,inplace=True)
data.company_size.value_counts()

In [ ]:
#Missing values
missing(data,'experience')

In [ ]:
# we will replace '<1' with 0 and '>20' with 21
#Missing values will be replaced mode which is '>20'
data.experience.fillna('>20',inplace=True)
data.experience.replace(['<1','>20'],[0,21],inplace=True)
data.experience=data.experience.astype('int64')


In [ ]:
data.experience.value_counts().sort_index()

In [ ]:
#Experience Plot
print('Fig 1.4')
plt.figure(figsize=(10,10))
plt.title('Experience_Plot',size=20)
sns.pointplot(x=[x for x in range(0,22)],y=data.experience.value_counts().sort_index().values,color='g')
plt.xticks([x for x in range(0,22)],['<1']+[str(x) for x in range(1,21)]+['>20'])
plt.xlabel('Experience',size=15)
plt.grid('True')
plt.show()

Candidates with 20+ years work experience accounts for much(3725 applicants) of the applicants. This group of candidates with much work experience can be said to be professionals. This can be seen from fig 1.3 plot above

In [ ]:
missing(data,'major_discipline')

In [ ]:
data.major_discipline.value_counts()

In [ ]:
#Pie Plot
print('Fig 1.5')
piechart('major_discipline',(15,10))

Major_Discipline contains 3125 missing points. STEM accounts for 88% of the Disciplines.Therefore missing points will be replaced with STEM

In [ ]:
data.major_discipline.fillna('STEM',inplace=True)
data.major_discipline.replace(['STEM','Humanities','Arts','Business Degree','No Major','Other'],[0,1,2,3,4,5],inplace=True)

In [ ]:
missing(data,'education_level')

In [ ]:
data.education_level.value_counts()

In [ ]:
#Fill Missing values in Education Level with Graduate
data.education_level.fillna('Graduate',inplace=True)

In [ ]:
# custom Encoding Of Education Level
data.education_level.replace(['Primary School','High School','Graduate','Masters','Phd'],[0,1,2,3,4],inplace=True)

In [ ]:
#Missing points
missing(data,'enrolled_university')

In [ ]:
#Pie Chart
print('Fig 1.6')
piechart('enrolled_university',(8,10))

In [ ]:
#Fill missing entries in enrolled data and  Custom encoding
data.enrolled_university.fillna('no_enrollment',inplace=True)
data.enrolled_university.replace(['no_enrollment','Part time course','Full time course'],[0,1,2],inplace=True)

In [ ]:
missing(data,'relevent_experience')

No Missing Value in Relevant Experience

In [ ]:
#Custom Encoding
data.relevent_experience.replace(['Has relevent experience','No relevent experience'],[0,1],inplace=True)

In [ ]:
data.relevent_experience.value_counts()

In [ ]:
data.head(5)

In [ ]:
missing(data,'gender')

In [ ]:
data.gender.value_counts()

In [ ]:
#Pie Plot
print('Fig 1.7')
piechart('gender',(10,10))

Male Gender Make up for 90% of the candidates

In [ ]:
#fill missing Gender points with Male
data.gender.fillna('Male',inplace=True)
data.gender.replace(['Female','Male','Other'],[0,1,2],inplace=True)


In [ ]:
#Missning points
missing(data,'city_development_index')

There are no missing entries in city development index column

In [ ]:
#Plot of City Development Index
print('Fig 1.8')
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(10,10))
fig.suptitle('City Development Index',size=20)
ax1.set_title('City_Index_Histplot',size=15)
ax1.grid('True')
ax1.set_xlabel('Index',size=10)
ax1.hist(data.city_development_index,color='g')
ax2.set_title('City_Index_Boxplot',size=15)
ax2.grid('True')
ax2.set_xlabel('Index',size=10)
sns.boxplot(y=data.city_development_index,ax=ax2)
plt.show()

Fig 1.8 above shows that,city devwlopment index is constrained between the values of 0 and 1. The boxplot shows that there are outliers 

In [ ]:
missing(data,'city')

In [ ]:
data.city.value_counts()

In [ ]:
# We extract the codes at the end of the entries in city example for 'city_140' we extract 140
city=data.city.values
b=[]
for x in city:
    b.append(x[-3:])
c=[]
for x in range(len(b)):
    if b[x].isdigit():
        c.append(x)
    else:
        b[x]=b[x].replace('_','')
        b[x]=b[x].replace('y','')
        c.append(b[x])
c=[int(x) for x in c]
c[:5]
data.city=c
data.head(5)

In [ ]:
#Length Of Unique Values in City
print('No Of Unique Cities: ',len(data.city.unique()))

In [ ]:
#City Plot
print('Fig 1.9')
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(10,10))
fig.suptitle('City_Plot',size=20)
ax1.set_title('Boxplots',size=15)
ax1.grid('True')
sns.boxplot(y=data.city,ax=ax1)
ax1.set_xlabel('city',size=10)
ax2.set_title('Violin Plot')
ax2.set_xlabel('City',size=10)
ax2.grid('True')
sns.violinplot(y=data.city,ax=ax2)
ax2.grid('True')
plt.show()

Fig 1.9 Is a plot of City Codes after extraction. The Boxplot on the right shows that There is a large range of values for city. It ranges from 0-20000 while it has a median of about 5000.Most of its values are within the range of 0-5000 as seen from the violin plot on the right side of fig 1.9.

In [ ]:
#The Log of the values for City will be Taken to reduce and restrain to the range of other features
data.city=np.log10(data.city+1)
data.head(5)

In [ ]:
train_data.target.value_counts()

In [ ]:
#Plot of Target
print('{:^30}'.format('Fig 2.0'))
plt.figure(figsize=(10,10))
plt.hist(y_train,color='r')
plt.xlabel('Target',size=15)
plt.ylabel('Count',size=15)
plt.title('Target_Hist_Plot',size=20)
plt.grid('True')
plt.show()

Fig 2.0 Above is a plot of Target values for the dataset if  consists of values 0 and 1 in an imblanced ratio of 4:1 in favour of the 0 class. This can be very problematic to our model if not handled because it make it to be skewed to the 0 class amd may label most points as zero due to this effect.

In [ ]:
data.head(5)

In [ ]:
#Split Data Back into test and Train
#Drop Id Column
data.drop('enrollee_id',axis=1,inplace=True)
train_data=data.iloc[:train_data.shape[0]]
test_data=data.iloc[train_data.shape[0]:]
train_data['target']=y_train
print('Train_Data_Shape: ',train_data.shape,'\n\n','Test_Data_Shape: ',test_data.shape)

In [ ]:
train_data.head(5)

In [ ]:
#HeatMap Of Correllations
plt.figure(figsize=(20,20))
corrmat=train_data.corr()
mask=np.array(corrmat)
mask[np.tril_indices_from(corrmat)]=False
sns.heatmap(corrmat,mask=mask,vmax=1.0,square=True,annot=True)
plt.title('Heatmap_Of_Correlations')
plt.show()

In [ ]:
#PCA
seed=42
pca=PCA(n_components=2,random_state=42)
pca.fit(train_data.drop('target',axis=1))
z=pca.transform(train_data.drop('target',axis=1))
pca.explained_variance_ratio_

From the explained variance ratio above of the Dimensionally reduced data. The first Principal component contains 98% of the information and sufficiently describes the whole data. This is and indication that the Features are linearly dependent and may have much colinearity among them.

In [ ]:
#Clustering
kmeans=KMeans(n_clusters=2,random_state=42)
kmeans.fit(train_data.drop('target',axis=1))
km=kmeans.transform(train_data.drop('target',axis=1))
labels=kmeans.labels_
centers=kmeans.cluster_centers_

In [ ]:
#Plot
z_data=pd.DataFrame({'z1':z[:,0],'z2':z[:,1],'target':y_train})
cluster_data=pd.DataFrame({'z1':km[:,0],'z2':km[:,1],'target':labels})
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(15,15))
fig.suptitle('Data_Plots')
ax1.set_title('PCA_Reduced_Scatter_Plot',size=15)
ax1.set_xlabel('Z1')
ax1.grid('True')
ax1.plot(z_data.z1[z_data.target==0],z_data.z2[z_data.target==0],'r.')
ax1.plot(z_data.z1[z_data.target==1],z_data.z2[z_data.target==1],'g.')
ax1.set_ylabel('Z2')
ax2.set_title('KMeans_Clustered_Scatter_Plot',size=15)
ax2.set_xlabel('Kmeans1');ax2.set_ylabel('KMeans2')
ax2.grid('True')
ax2.plot(cluster_data.z1[cluster_data.target==0],cluster_data.z2[cluster_data.target==0],'r.')
ax2.plot(cluster_data.z1[cluster_data.target==1],cluster_data.z2[cluster_data.target==1],'g.')
plt.show()
                                                      

### MODEL BUILDING AND PREDICTION

We Will Use Roc_Auc as the metric for Optimization and Evaluation of our Models. We will also we predicting Probabilities instead of Actual Classes. We will also be Using Cross Validation To Prevent Overfitting

In [ ]:
#STRATIFIED SHUFFLE SPLIT
split=StratifiedShuffleSplit(n_splits=1,test_size=0.2,random_state=42)
for x,y in split.split(train_data,train_data.target):
    x_train=train_data.loc[x]
    val_test=train_data.loc[y]
y_train=x_train.target.values
y_val=val_test.target.values
x_train.drop('target',axis=1,inplace=True)
val_test.drop('target',axis=1,inplace=True)
print("X_Train: ",x_train.shape," \n\n",'Validation_Test: ',val_test.shape)

In [ ]:
#STANDARD SCALING
scaler=StandardScaler()
scaler.fit(data)
x_train_scaled=scaler.transform(x_train)
val_test_scaled=scaler.transform(val_test)
test_scaled=scaler.transform(test_data)
print('Train_Data_Scaled: ',x_train_scaled.shape,'\n\n','Test_Data :',test_scaled.shape)


In [ ]:
#Function For Evaluating Metrics
def score(model,x,y,Method):
    pred=cross_val_predict(model,x,y,cv=5)
    pred_proba=cross_val_predict(model,x,y,cv=5,method=Method)[:,1]
    print('Confusion_Matrix: ','\n\n',confusion_matrix(y,pred))
    print('Precision_Score: ',round(precision_score(y,pred),3),'\n\n','Recall_Score: ',round(recall_score(y,pred),3))
    print('Roc_Auc_Score: ',round(roc_auc_score(y,pred_proba),3))
    return pred,pred_proba
          
          

In [ ]:
#ROC PLOT FUNCTION
def roc_plot(algorithm,y_true,y_pred_proba):
    fpr,tpr,thresholds=roc_curve(y_true,y_pred_proba)
    plt.figure(figsize=(10,10))
    plt.grid('True')
    plt.title(algorithm.capitalize()+' Roc Plot',size=20)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.plot(fpr,tpr,'r-',lw=2)
    plt.axis([0,1,0,1])
    plt.plot([0,1],[0,1],'g--')
    plt.show()

In [ ]:
log_clf=LogisticRegression(C=1,random_state=42)

In [ ]:
log_pre,log_proba=score(log_clf,x_train_scaled,y_train,'predict_proba')

In [ ]:
roc_plot('Logistic',y_train,log_proba)

In [ ]:
#Test On Validation Data
log_pre,log_proba=score(log_clf,val_test_scaled,y_val,'predict_proba')

Logistic Regression Gives An Roc_Score of 0.76 on Validation Data

In [ ]:
#HyperParameter Tuning
tree_clf=DecisionTreeClassifier(random_state=42)
grid_params=[{'max_leaf_nodes':[10,20,35,50,100],'min_samples_split':[5,10,20,50],'min_samples_leaf':[5,10,20,30,35,50]}]
grid=GridSearchCV(tree_clf,grid_params,cv=5,scoring='roc_auc')
grid.fit(val_test,y_val)

In [ ]:
cv=grid.cv_results_
cv=pd.DataFrame(cv)
print('Best_Grid_Score: ',grid.best_score_,'\n\n','Best_Grid_Params: ',grid.best_params_)
#Plot Of Grid Results
fig,ax1=plt.subplots(figsize=(10,10))
ax1.grid('True')
plt.title('Grid Search Mean and Std Score Plot')
ax1.plot(cv.index,cv['mean_test_score'],'g*-',lw=3,label='Mean_Test_Score')
ax2=ax1.twinx()
ax2.plot(cv.index.values,cv['std_test_score'],'r*-',lw=3,label='Std_Test_Score')
plt.xlabel('Index_No')
fig.tight_layout()
plt.legend(loc='best')
plt.show()

The plot above represents The mean test score and Std test Score for Grid Search on decision Tree Classifier. The points represents The Mean test Csore While Std Test Score Represnts Std Test Score.
Our Goal is to Get the best score values and the lowest Std test score. Between 80-90 we achieve our Best Test Score and a reasonably low Std Score.

In [ ]:
tree_clf=DecisionTreeClassifier(random_state=42,max_leaf_nodes=50,min_samples_split=5,min_samples_leaf=35)
tree_clf.fit(x_train,y_train)


In [ ]:
tree_pre,tree_proba=score(tree_clf,x_train,y_train,'predict_proba')

In [ ]:
roc_plot('Decision Tree',y_train,tree_proba)

In [ ]:
#Validation Test
score(tree_clf,val_test,y_val,'predict_proba')

The Decision Tree Classifier Performs Relatively Better than Our Earlier Logistic Regression Algorithm with a score of 0.799

In [ ]:
#Gradeint Boosting
grad_clf=GradientBoostingClassifier(random_state=42,max_leaf_nodes=50,min_samples_leaf=35,min_samples_split=5)
grad_clf.fit(x_train,y_train)

In [ ]:
score(grad_clf,x_train,y_train,'predict_proba')

In [ ]:
#Gradient Boosting Classifier on Validation Test Data
score(grad_clf,val_test,y_val,'predict_proba')

Gradient Boosting Classifier Acheives an Roc_Auc_Score of 0.812 on Validation Test

## GAINING INSIGHTS INTO WHY CANDIDATES DECIDES TO STAY OR LEAVE THE COMPANY USING THE MODELS.

In [ ]:
x_train.head(5)

In [ ]:
#Gradient Boosting Feature Importances
z_grad=pd.DataFrame()
z_grad['Features']=data.columns.values
z_grad['Importance_Rank']=grad_clf.feature_importances_
z_grad=z_grad.sort_values(by='Importance_Rank',ascending=False)
z_grad

The Most Important Features Used in Gradient Boosting Regressor are City_Devlopment_Index,Company Size,Education Level,Experience and  Last New in descending oreder of less importance

In [ ]:
#Decision Tree Feature Importances
z_tree=pd.DataFrame()
z_tree['Features']=data.columns
z_tree['Importance_Rank']=tree_clf.feature_importances_
z_tree=z_tree.sort_values(by='Importance_Rank',ascending=False)
z_tree

For Decision Tree Classifier The Most Importance Geatures for Differntiating Between  Candidates who left after Training and those who remained Where City_Development_Index,Company_Size,Education Level,City,Relevant Experience and Experience

We will Use RFE Wrapper to Get a Clue of The most Important Features For Logistic Regressiin

In [ ]:
from sklearn.feature_selection import RFE

In [ ]:
rfe=RFE(log_clf,n_features_to_select=4)
rfe.fit(x_train,y_train)
rfe.ranking_
z_log=pd.DataFrame()
z_log['Features']=data.columns
z_log['Importance_Rank']=rfe.ranking_
z_log.index.name='Index'
z_log=z_log.sort_values(ascending=True,by='Importance_Rank')
z_log

For Our Logistic Regression Classifier The most Importance Features for Decision Making where City_Development_Index,Education Level,Experience, Company Size

In [ ]:
print('Fig 3.0')
plt.figure(figsize=(20,20))
plot_tree(tree_clf,feature_names=data.columns.values,fontsize=6,rounded=True,class_names=['remain','quit'],filled=True)
plt.show()

Fig 3.0 Above Is a graphical plot of the decision tree used for prediction. RFFE wrapper was used to get the features from our Logistic Regression. 

###### So What Can We Say About Our Features?.

1. The Singular Most Important Feature For Decision Making according to our Models Was City Development Index. The Tree Plot on Fig 3.0 Shows us that Candidates with city development index less than 0.625 where more likely to remain more than those with city  development index. Which Means the decisuon by Candidates to Remain in the company was inversely Proportional to the City Development index.


2. Other Features Necessary for Predicting The probabilities of Candidates to Stay Back at The Company or to seek Jobs else where were Education Level Experience or Company Size.

 3.3. Candidates who did not indicate the size of their previous workplace where likely to to remain

A closer study of Fig Will Give Relationship Among These Important Features

In [ ]:
#Predicting Test Data
sub_file=sample_submission
sub_file['target']=grad_clf.predict_proba(test_data)
fh=sub_file.to_csv('Submission_File.csv',index=False)
sub_file.tail(10)